In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width: 100% !important; }</style>"))

In [2]:
CONSTRAINT_TOOLS_DATA = '/scratch/ucgd/lustre-work/quinlan/data-shared/constraint-tools'

TRAIN_SET_LABEL = 'noncoding'

In [3]:
import pandas as pd 

def get_observations_and_mchale_predictions_on_chen_windows(kmer_size, train_set_label, noisy_label):
    # zscores were computed using dist/model-germline-grch38-Nonly.kmerSize-X.trainSet-Y{Z}.json                                                                  
    filename = f'{CONSTRAINT_TOOLS_DATA}/genome-wide-predictions/predict-germline-grch38-Nonly.chenWindows.kmerSize-{kmer_size}.trainSet-{train_set_label}{noisy_label}.bed'
    df = pd.read_csv(filename, sep='\t')
    return df

get_observations_and_mchale_predictions_on_chen_windows(kmer_size=3, train_set_label=TRAIN_SET_LABEL, noisy_label='.noisified')

,chromosome,start,end,position,N_bar,N_observed,N_mean_null,N_variance_null
0,chr1,1432000,1433000,1432500,-0.283079,260,264.038617,203.540686
1,chr1,1435000,1436000,1435500,-7.609011,275,389.032484,224.595337
2,chr1,1449000,1450000,1449500,0.225283,225,221.996472,177.748203
3,chr1,1450000,1451000,1450500,2.147648,313,282.634862,199.905291
4,chr1,1451000,1452000,1451500,2.332657,293,260.580640,193.155585
...,...,...,...,...,...,...,...,...
1796995,chr22,50224000,50225000,50224500,0.425725,254,248.099121,192.121344
1796996,chr22,50227000,50228000,50227500,1.182923,283,266.223120,201.144968
1796997,chr22,50228000,50229000,50228500,-0.349426,241,245.934204,199.398832
1796998,chr22,50229000,50230000,50229500,-1.355873,231,249.853718,193.355386


In [4]:
def create_new_column(old_column, kmer_size, train_set_label, noisy_label): 
    new_column = f'{old_column}_{kmer_size}_{train_set_label}'
    if noisy_label == '.noisified': new_column += '_noisy'
    return new_column
    
def get_mchale_predictions_on_chen_windows(kmer_size, train_set_label, noisy_label):
    old_columns = ['N_bar', 'N_mean_null', 'N_variance_null']
    df = get_observations_and_mchale_predictions_on_chen_windows(kmer_size, train_set_label, noisy_label)[['chromosome', 'start', 'end'] + old_columns]
    df = df.rename(columns={
        old_column: create_new_column(old_column, kmer_size, train_set_label, noisy_label)
        for old_column in old_columns
    })
    return df

def get_observations_on_chen_windows(kmer_size, train_set_label, noisy_label):
    columns = ['chromosome', 'start', 'end', 'N_observed']
    df = get_observations_and_mchale_predictions_on_chen_windows(kmer_size, train_set_label, noisy_label)[columns]
    return df

get_mchale_predictions_on_chen_windows(kmer_size=3, train_set_label=TRAIN_SET_LABEL, noisy_label='.noisified')

,chromosome,start,end,N_bar_3_noncoding_noisy,N_mean_null_3_noncoding_noisy,N_variance_null_3_noncoding_noisy
0,chr1,1432000,1433000,-0.283079,264.038617,203.540686
1,chr1,1435000,1436000,-7.609011,389.032484,224.595337
2,chr1,1449000,1450000,0.225283,221.996472,177.748203
3,chr1,1450000,1451000,2.147648,282.634862,199.905291
4,chr1,1451000,1452000,2.332657,260.580640,193.155585
...,...,...,...,...,...,...
1796995,chr22,50224000,50225000,0.425725,248.099121,192.121344
1796996,chr22,50227000,50228000,1.182923,266.223120,201.144968
1796997,chr22,50228000,50229000,-0.349426,245.934204,199.398832
1796998,chr22,50229000,50230000,-1.355873,249.853718,193.355386


In [5]:
from functools import reduce

on = ['chromosome', 'start', 'end']
how = 'inner'

dfs = [get_observations_on_chen_windows(kmer_size=3, train_set_label=TRAIN_SET_LABEL, noisy_label='')]

dfs.append(get_mchale_predictions_on_chen_windows(kmer_size=7, train_set_label=TRAIN_SET_LABEL, noisy_label=''))

for kmer_size in [1, 3, 5]: 
    for train_set_label in [TRAIN_SET_LABEL]: 
        for noisy_label in ['', '.noisified']:
            dfs.append(get_mchale_predictions_on_chen_windows(kmer_size, train_set_label, noisy_label))

mchale_predictions = reduce(
    lambda left, right: pd.merge(left, right, on=on, how=how), 
    dfs
)

mchale_predictions

,chromosome,start,end,N_observed,N_bar_7_noncoding,N_mean_null_7_noncoding,N_variance_null_7_noncoding,N_bar_1_noncoding,N_mean_null_1_noncoding,N_variance_null_1_noncoding,...,N_variance_null_3_noncoding,N_bar_3_noncoding_noisy,N_mean_null_3_noncoding_noisy,N_variance_null_3_noncoding_noisy,N_bar_5_noncoding,N_mean_null_5_noncoding,N_variance_null_5_noncoding,N_bar_5_noncoding_noisy,N_mean_null_5_noncoding_noisy,N_variance_null_5_noncoding_noisy
0,chr1,1432000,1433000,260,-0.372778,265.321432,203.777753,1.590921,236.519249,217.834403,...,203.072610,-0.283079,264.038617,203.540686,-0.339806,264.852332,203.910737,-0.340167,264.858613,204.004160
1,chr1,1435000,1436000,275,-6.280673,368.627875,222.227572,2.097960,243.596443,224.059444,...,223.937877,-7.609011,389.032484,224.595337,-6.810107,375.796139,219.068492,-6.784165,375.458319,219.269877
2,chr1,1449000,1450000,225,0.139977,223.137143,177.111728,0.959620,211.569144,195.888493,...,177.372370,0.225283,221.996472,177.748203,0.330975,220.612316,175.743926,0.330023,220.624461,175.781765
3,chr1,1450000,1451000,313,2.524948,277.859484,193.691881,5.870332,227.882674,210.237742,...,199.400099,2.147648,282.634862,199.905291,2.480911,278.322530,195.376527,2.482902,278.288535,195.445943
4,chr1,1451000,1452000,293,2.951595,252.829443,185.226049,4.803930,223.924244,206.755939,...,192.772432,2.332657,260.580640,193.155585,2.725116,255.650992,187.839972,2.724052,255.656435,187.931976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1796995,chr22,50224000,50225000,254,0.557227,246.341222,188.909706,2.153889,223.084577,206.017374,...,191.722062,0.425725,248.099121,192.121344,0.496005,247.169019,189.668194,0.493078,247.208571,189.710211
1796996,chr22,50227000,50228000,283,1.371285,263.752490,197.012560,3.418013,232.920676,214.669127,...,200.707864,1.182923,266.223120,201.144968,1.340543,264.132772,198.086499,1.334261,264.214596,198.225408
1796997,chr22,50228000,50229000,241,-0.330285,245.639165,197.288944,0.518173,233.400486,215.091164,...,198.944848,-0.349426,245.934204,199.398832,-0.348897,245.916863,198.601635,-0.357060,246.034244,198.786653
1796998,chr22,50229000,50230000,231,-1.225998,247.885488,189.691434,0.509033,223.684339,206.544920,...,192.937946,-1.355873,249.853718,193.355386,-1.250236,248.270789,190.827211,-1.246767,248.224941,190.873186


In [6]:
mchale_predictions.to_csv(
    f'{CONSTRAINT_TOOLS_DATA}/benchmark-genome-wide-predictions/chen-et-al-2022/mchale.kmerSizes.trainSets.noisy.bed', 
    sep = '\t',
    index = False
)